In [79]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [87]:
file_path = "data_all_teams_gdp.csv"
df_gdp = pd.read_csv(file_path, delimiter=",")  

print(df_gdp.head())

   year        gdp
0     0  947610.99
1     1  931427.95
2     2  872704.64
3     3  875281.14
4     4  853371.77


In [88]:
file_path = "data_all_teams.csv"
df = pd.read_csv(file_path, delimiter=",")  

print(df.head())

   year sex     profession   income
0     0   F  civil servant  5131.08
1     0   M          child     0.00
2     0   M     unemployed   124.10
3     0   M         farmer  1868.39
4     0   F          child     0.00


In [89]:
df['avg_income_per_profession'] = df.groupby(['year', 'profession'])['income'].transform('mean')



In [92]:
df.to_csv('file1.csv')

In [82]:
import numpy as np
import pandas as pd

# Gender Distribution Calculation
gender_distribution = df.groupby(['year', 'sex']).size().unstack(fill_value=0)
gender_distribution['total'] = gender_distribution.sum(axis=1)
gender_distribution['female_percentage'] = gender_distribution['F'] / gender_distribution['total']

# Employment Rate Calculation
non_employed_professions = ['child', 'unemployed', 'homemaker', 'retired']
df['employed'] = ~df['profession'].isin(non_employed_professions)
employment_rate = df.groupby('year')['employed'].mean()  # Convert to percentage

# Gini Coefficient Calculation
def gini_coefficient(income_list):
    """Computes the Gini coefficient of a list of incomes."""
    incomes = np.sort(income_list)  # Sort incomes in ascending order
    n = len(incomes)
    if n == 0:
        return np.nan  # Avoid division by zero
    cumulative_income_sum = np.sum((np.arange(1, n + 1) * incomes))
    total_income_sum = np.sum(incomes)
    
    gini = (2 * cumulative_income_sum) / (n * total_income_sum) - (n + 1) / n
    return gini

gini_per_year = df.groupby('year')['income'].apply(gini_coefficient)







,index,female_percentage,employment_rate,gini_coefficient,average_income
0,0,0.468000,0.516000,0.604055,0.000000
1,1,0.468379,0.507905,0.607001,5357.960465
2,2,0.468627,0.498039,0.634642,4212.391458
3,3,0.466797,0.492188,0.613444,2182.726575
4,4,0.469786,0.483431,0.625739,2232.419245


In [84]:
df = df.merge(average_income, on='profession', how='left')



In [86]:

label_encoder = LabelEncoder()
df['sex'] = label_encoder.fit_transform(df['sex'])  # F -> 0, M -> 1
df['profession'] = label_encoder.fit_transform(df['profession'])

# Selecting features (X) and target (y)
X = df[['year', 'sex', 'profession', 'income']]
y = df['gdp']

# Splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model: Random Forest Regressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


# Predict GDP for the next 5 years
years_to_predict = np.array([1, 2, 3, 4, 5]).reshape(-1, 1)  # Next 5 years

predicted_gdp = model.predict(np.hstack([years_to_predict, np.full((5, 1), 1), np.full((5, 1), 1), np.full((5, 1), 1000)]))  # Example values for 'sex', 'profession', and 'income'

# Output predictions
for year, gdp in zip(range(1, 6), predicted_gdp):
    print(f"Predicted GDP for Year {year}: {gdp:.2f}")

# Feature Importance to see which factors contribute most to GDP prediction
feature_importances = model.feature_importances_
feature_names = X.columns

# Plotting feature importance
plt.barh(feature_names, feature_importances)
plt.xlabel("Feature Importance")
plt.title("Feature Importance in Predicting GDP")
plt.show()

KeyError: "['year', 'income'] not in index"

In [ ]:
y_pred = model.predict(X_test)
print(f"Mean Absolute Error: {mean_absolute_error(y_test, y_pred)}")
